In [12]:
data = pd.read_csv("my_datacleaned_file.tsv", sep='\t')
data

,Review,Positive/Negative rating,Cleaned_Review
0,Review,Liked,Review
1,Wow... Loved this place.,1,Wow Loved this place
2,Crust is not good.,0,Crust is not good
3,Not tasty and the texture was just nasty.,0,Not tasty and the texture was just nasty
4,Stopped by during the late May bank holiday of...,1,Stopped by during the late May bank holiday of...
...,...,...,...
996,I think food should have flavor and texture an...,0,I think food should have flavor and texture an...
997,Appetite instantly gone.,0,Appetite instantly gone
998,Overall I was not impressed and would not go b...,0,Overall I was not impressed and would not go b...
999,"The whole experience was underwhelming, and I ...",0,The whole experience was underwhelming and I ...


In [13]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("my_datacleaned_file.tsv", sep='\t')
data.columns = ['Review', 'Positive/Negative rating', 'Cleaned_Review']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100 #output number punct as percentage of total words in review

data['body_len'] = data['Review'].apply(lambda x: len(x) - x.count(" ")) #function to count number of words in original reviews
data['punct%'] = data['Review'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[['Review', 'body_len', 'punct%']], data['Positive/Negative rating'], test_size=0.2)

In [15]:
data_vector = TfidfVectorizer(analyzer=clean_text) #  'char', 'char_wb', 'word', or a callable function.
data_vector_fit = data_vector.fit(X_train["Review"])

## error on parameters

#fixed by putting function of text with removed punctuation into analyzer - 9/12

data_train = data_vector_fit.transform(X_train["Review"])
data_test = data_vector_fit.transform(X_test["Review"])

#accepts list of object - connect data based on indicies 
#turn matrix to dataframe
#put data in dataframe side by side
X_train_vect = pd.concat([X_train[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(data_train.toarray())], axis=1)
X_test_vect = pd.concat([X_test[['body_len', 'punct%']].reset_index(drop=True), 
           pd.DataFrame(data_test.toarray())], axis=1)

X_train_vect.head()

#some words are not recogized to be vectorized which is ok

,body_len,punct%,0,1,2,3,4,5,6,7,...,1465,1466,1467,1468,1469,1470,1471,1472,1473,1474
0,31,9.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.612236,0.0,0.0,0.0,0.0,0.0
1,26,3.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,40,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,104,1.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,68,2.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

# Convert column names to strings in X_train_vect and X_test_vect, must put before code
X_train_vect.columns = X_train_vect.columns.astype(str)
X_test_vect.columns = X_test_vect.columns.astype(str)

# Initialize the RandomForestClassifier
my_rf = RandomForestClassifier(n_estimators=155, max_depth=None, n_jobs=-1)

# Train the model
my_rf_model = my_rf.fit(X_train_vect, y_train)
my_y_pred = my_rf_model.predict(X_test_vect)

# Calculate precision, recall, and fscore
precision, recall, fscore, train_support = precision_recall_fscore_support(y_test, my_y_pred, pos_label="1", average="binary")

# Calculate accuracy
accuracy = (my_y_pred == y_test).sum() / len(my_y_pred)

# Output results in categorizing whether reviews are positive or negative reviews
print("Precision: {} | Recall: {} | Accuracy: {}".format(
    round(precision, 3), round(recall, 3), round(accuracy, 3)))

print("As a percentage: {}% | {}% | {}%".format(
    precision*100, recall*100, accuracy*100))
    




Precision: 0.711 | Recall: 0.596 | Accuracy: 0.682
As a percentage: 71.08433734939759% | 59.59595959595959% | 68.1592039800995%
